<a href="https://colab.research.google.com/github/Nix07/Utilizing-BERT-for-Aspect-Based-Sentiment-Analysis/blob/master/InferenceAndEvaluationonSentiHood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers

Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 778kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 23.5MB/s 
     |████████████████████████████████| 890kB 24.6MB/s 
     |████████████████████████████████| 3.0MB 38.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=213c561070f02cb0284bb96340ec97a72fbf80277660f7a5ff5dbe98711011bf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import transformers
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertModel, BertTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

# Making Predictions on Testing Set

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, BERT_MODEL):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids, device, tokenizer=None):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
class SentiHood:
  def __init__(self, opinions_id, text, targets, tokenizer, max_len):
    self.opinions_id = opinions_id
    self.text = text
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, item):
    opinions_id = self.opinions_id[item]
    text = str(self.text[item])
    targets = self.targets[item]

    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_len,
        pad_to_max_length = True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids": torch.tensor(ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        "targets": torch.tensor(targets, dtype=torch.long),
        "opinions_id": torch.tensor(opinions_id, dtype=torch.long)
    }

In [ ]:
def infer_loop_function(data_loader, model, device, tokenizer, location, aspect):
  model.eval()

  df_pred = pd.DataFrame({"id": [], "predicted": [], "actual": []})

  ii = 0
  for bi, d in tqdm(enumerate(data_loader), total=len(data_loader), ncols=80, leave=False):
    opinions_id = d["opinions_id"]
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    opinions_id = opinions_id.to(device, dtype=torch.long)
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, device=device, tokenizer=tokenizer)
    _, predicted = torch.max(outputs, 1)
    
    predicted = predicted.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    opinions_id = opinions_id.detach().cpu().numpy()

    for k in range(len(predicted)):
      df_pred.loc[ii] = [str(opinions_id[k]), str(predicted[k]), str(targets[k])]
      ii += 1

  print(f"{location}{aspect} DONE!")
  save_path = '/content/drive/My Drive/SentiHood/TestingData/Predicted' + str(location) + str(aspect) + '.csv'
  df_pred.to_csv(save_path, index=False)


In [ ]:
def run():
  MAX_LEN = 140
  BATCH_SIZE = 16
  BERT_MODEL = 'bert-base-uncased'

  locations = ['LOCATION1', 'LOCATION2']
  aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

  for location in locations:
    for aspect in aspects:
      print(f"Starting {location} {aspect}...")
      testing_set_path = '/content/drive/My Drive/SentiHood/TestingData/' + str(location) + str(aspect) + '.csv'

      df_test = pd.read_csv(testing_set_path)
      sentiment_mapping = {
          'Positive': 0,
          'Negative': 1,
          'None': 2
      }
      df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
      df_test = df_test.reset_index(drop=True)

      tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

      test_dataset = SentiHood(
          opinions_id = df_test['id'].values,
          text = df_test['text'].values,
          targets = df_test['sentiment'].values,
          tokenizer = tokenizer,
          max_len = MAX_LEN
      )

      test_data_loader = torch.utils.data.DataLoader(
          test_dataset,
          batch_size = BATCH_SIZE,
          shuffle = False
      )

      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      print(f"Device: {device}")

      model = torch.load('/content/drive/My Drive/SentiHood/LocationAspectPairs(NEW)/'+str(location)+str(aspect)+'/best.bin')
      infer_loop_function(data_loader=test_data_loader, model=model, device=device, tokenizer=tokenizer, location=location, aspect=aspect)

if __name__ == "__main__":
  run()     

Starting LOCATION1 dining...
Device: cuda:0


LOCATION1dining DONE!
Starting LOCATION1 general...


Device: cuda:0
LOCATION1general DONE!
Starting LOCATION1 green-nature...
Device: cuda:0


LOCATION1green-nature DONE!
Starting LOCATION1 live...
Device: cuda:0


LOCATION1live DONE!
Starting LOCATION1 multicultural...
Device: cuda:0


LOCATION1multicultural DONE!
Starting LOCATION1 nightlife...
Device: cuda:0


LOCATION1nightlife DONE!
Starting LOCATION1 price...
Device: cuda:0


 77%|████████████████████████████████▉          | 72/94 [00:23<00:07,  3.12it/s]

#  Evaluation

### Creating dataframe containing true labels of the testing set

In [6]:
df_true_location1 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/TestingData/LOCATION1' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  sentiment_mapping = {
      'Positive': 0,
      'Negative': 1,
      'None': 2
  }
  df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
  df_test = df_test.reset_index(drop=True)

  df_true_location1[aspect] = df_test['sentiment']

df_true_location1['location'] = 'LOCATION1'
df_true_location1['id'] = df_test['id']

In [7]:
df_true_location2 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/TestingData/LOCATION2' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  sentiment_mapping = {
      'Positive': 0,
      'Negative': 1,
      'None': 2
  }
  df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
  df_test = df_test.reset_index(drop=True)

  df_true_location2[aspect] = df_test['sentiment']

df_true_location2['location'] = 'LOCATION2'
df_true_location2['id'] = df_test['id']

In [8]:
df_true = pd.concat([df_true_location1, df_true_location2])
df_true

,id,location,dining,general,green-nature,live,multicultural,nightlife,price,quiet,safety,shopping,touristy,transit-location
0,153,LOCATION1,2,2,2,2,2,2,2,2,0,2,2,2
1,1130,LOCATION1,2,0,2,2,2,2,2,2,0,2,2,2
2,1271,LOCATION1,2,1,2,2,2,2,0,2,2,2,2,2
3,1089,LOCATION1,2,1,2,2,2,2,2,2,2,2,2,2
4,731,LOCATION1,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,1431,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
384,1290,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
385,363,LOCATION2,2,2,2,2,2,2,2,2,2,2,2,2
386,1304,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2


### Creating dataframe containing predicted labels of the testing set

In [9]:
df_predicted_location1 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/PredictedData/PredictedLOCATION1' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  df_test = df_test.reset_index(drop=True)

  df_predicted_location1[aspect] = df_test['predicted']

df_predicted_location1['location'] = 'LOCATION1'
df_predicted_location1['id'] = df_test['id']

In [10]:
df_predicted_location2 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/PredictedData/PredictedLOCATION2' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  df_test = df_test.reset_index(drop=True)

  df_predicted_location2[aspect] = df_test['predicted']

df_predicted_location2['location'] = 'LOCATION1'
df_predicted_location2['id'] = df_test['id']

In [11]:
df_predicted = pd.concat([df_predicted_location1, df_predicted_location2])
df_predicted

,id,location,dining,general,green-nature,live,multicultural,nightlife,price,quiet,safety,shopping,touristy,transit-location
0,153,LOCATION1,2,2,2,2,2,2,2,2,0,2,2,2
1,1130,LOCATION1,2,0,2,2,2,2,2,2,0,2,2,2
2,1271,LOCATION1,2,1,2,2,2,2,1,2,2,2,2,2
3,1089,LOCATION1,2,0,2,2,2,2,2,2,2,2,2,2
4,731,LOCATION1,2,2,2,2,2,2,2,2,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,1431,LOCATION1,2,0,2,2,2,2,2,2,2,2,2,2
384,1290,LOCATION1,2,0,2,2,2,2,2,2,2,2,2,2
385,363,LOCATION1,2,2,2,2,2,2,2,2,2,2,2,2
386,1304,LOCATION1,2,2,2,2,2,2,1,2,2,2,2,2


**Aspect Accuracy**

In [38]:
def compute_aspect_accuracy(df_true, df_predicted):
  df_true = df_true.replace([0, 1], 1).replace(2, 0)
  df_predicted = df_predicted.replace([0, 1], 1).replace(2, 0)

  count = 0
  total = 0

  for i in range(df_true.shape[0]):
    true_values = df_true.iloc[i].values[2:]
    predicted_values = df_predicted.iloc[i].values[2:]

    if (true_values == predicted_values).all():
      count += 1
    total += 1

  accuracy = float(count)/float(total)*100
  return round(accuracy, 2)

In [39]:
print(f"Aspect Accuracy (strict): {compute_aspect_accuracy(df_true, df_predicted)}%")

Aspect Accuracy (strict): 68.6%


**Aspect F1 Score**

In [44]:
def compute_aspect_f1_score(df_true, df_predicted):
  df_true = df_true.replace([0, 1], 1).replace(2, 0)
  df_predicted = df_predicted.replace([0, 1], 1).replace(2, 0)

  total_f1_score = 0
  total = 0

  for i in range(df_true.shape[0]):
    true_values = list(df_true.iloc[i].values[2:])
    predicted_values = list(df_predicted.iloc[i].values[2:])

    total_f1_score += f1_score(true_values, predicted_values, average="macro")
    total += 1

  score = float(total_f1_score)/float(total)*100
  return round(score, 2)

In [45]:
print(f"Aspect F1 score: {compute_aspect_f1_score(df_true, df_predicted)}")

Aspect F1 score: 88.19


**Sentiment Accuracy**

In [46]:
def compute_sentiment_accuracy(df_true, df_predicted):
  """3-class Classfication Accuracy"""

  count = 0
  total = 0

  for aspect in ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']:
    count += np.sum(df_true[aspect].values == df_predicted[aspect].values)
    total += df_true.shape[0]

  accuracy = float(count)/float(total) * 100
  return round(accuracy, 2)

In [47]:
print(f"Sentiment Accuracy: {compute_sentiment_accuracy(df_true, df_predicted)}")

Sentiment Accuracy: 96.45
